<a href="https://colab.research.google.com/github/ammadidrees786/section-wise-Paper-Recomendation-System-/blob/main/validpdf300.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.3 MB/s eta 0:00:00


In [6]:
import os
import requests
from PyPDF2 import PdfReader

os.makedirs("valid_pdfs", exist_ok=True)

base_url = "https://aclanthology.org/"
prefix = "2024.acl-long."
valid_pdfs = []
count = 0
max_valid = 300

for i in range(1, 10000):  # try first 1000 files
    if count >= max_valid:
        break

    paper_id = f"{i:03}"
    filename = f"{paper_id}_{prefix}{i}.pdf"
    url = base_url + f"{prefix}{i}.pdf"
    filepath = os.path.join("valid_pdfs", filename)

    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            with open(filepath, "wb") as f:
                f.write(response.content)

            # Check extractable text
            reader = PdfReader(filepath)
            text = ""
            for page in reader.pages[:4]:  # Check first 4 pages
                text += page.extract_text() or ""

            if text.strip():
                valid_pdfs.append(filename)
                count += 1
                print(f"✅ Valid: {filename}")
            else:
                os.remove(filepath)
                print(f"⚠️ Skipped (no text): {filename}")
        else:
            print(f"❌ Download failed: {filename}")
    except Exception as e:
        print(f"❌ Error {filename}: {e}")

print(f"\n✅ Total valid PDFs: {len(valid_pdfs)}")


✅ Valid: 001_2024.acl-long.1.pdf
✅ Valid: 002_2024.acl-long.2.pdf
✅ Valid: 003_2024.acl-long.3.pdf
✅ Valid: 004_2024.acl-long.4.pdf
✅ Valid: 005_2024.acl-long.5.pdf
✅ Valid: 006_2024.acl-long.6.pdf
✅ Valid: 007_2024.acl-long.7.pdf
✅ Valid: 008_2024.acl-long.8.pdf
✅ Valid: 009_2024.acl-long.9.pdf
✅ Valid: 010_2024.acl-long.10.pdf
✅ Valid: 011_2024.acl-long.11.pdf
✅ Valid: 012_2024.acl-long.12.pdf
✅ Valid: 013_2024.acl-long.13.pdf
✅ Valid: 014_2024.acl-long.14.pdf
✅ Valid: 015_2024.acl-long.15.pdf
✅ Valid: 016_2024.acl-long.16.pdf
✅ Valid: 017_2024.acl-long.17.pdf
✅ Valid: 018_2024.acl-long.18.pdf
✅ Valid: 019_2024.acl-long.19.pdf
✅ Valid: 020_2024.acl-long.20.pdf
✅ Valid: 021_2024.acl-long.21.pdf
✅ Valid: 022_2024.acl-long.22.pdf
✅ Valid: 023_2024.acl-long.23.pdf
✅ Valid: 024_2024.acl-long.24.pdf
✅ Valid: 025_2024.acl-long.25.pdf
✅ Valid: 026_2024.acl-long.26.pdf
✅ Valid: 027_2024.acl-long.27.pdf
✅ Valid: 028_2024.acl-long.28.pdf
✅ Valid: 029_2024.acl-long.29.pdf
✅ Valid: 030_2024.acl-l

In [8]:
import shutil

# Replace with the actual folder where your PDFs are saved
folder_to_zip = "/content/valid_pdfs"
zip_filename = "/content/valid_pdfs.zip"

shutil.make_archive(base_name=zip_filename.replace(".zip", ""), format='zip', root_dir=folder_to_zip)
print("✅ Folder zipped successfully.")


✅ Folder zipped successfully.


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import shutil

# Define destination path inside Google Drive
drive_destination = "/content/drive/MyDrive/valid_pdfs.zip"

# Copy the zip file
shutil.copy("/content/valid_pdfs.zip", drive_destination)

print(f"✅ ZIP file saved to: {drive_destination}")


✅ ZIP file saved to: /content/drive/MyDrive/valid_pdfs.zip


In [12]:
# List extracted PDFs
import os

pdf_list = os.listdir("valid_pdfs")
print(f"📄 Total extracted PDFs: {len(pdf_list)}")
print(pdf_list[:10])  # Show first 10 files


📄 Total extracted PDFs: 300
['093_2024.acl-long.93.pdf', '149_2024.acl-long.149.pdf', '273_2024.acl-long.273.pdf', '028_2024.acl-long.28.pdf', '150_2024.acl-long.150.pdf', '051_2024.acl-long.51.pdf', '126_2024.acl-long.126.pdf', '054_2024.acl-long.54.pdf', '033_2024.acl-long.33.pdf', '022_2024.acl-long.22.pdf']


In [13]:
!pip install lxml pdfminer.six

import os
from pdfminer.high_level import extract_text
import re
import pandas as pd


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 37.5 MB/s eta 0:00:00


In [14]:
# IMRaD-based headings mapping
imrad_map = {
    'Introduction': ['introduction', 'background', 'literature review'],
    'Methodology': ['methodology', 'method', 'design'],
    'Results': ['results', 'findings'],
    'Discussion': ['discussion', 'conclusion', 'analysis']
}


In [15]:
def extract_sections(text):
    sections = {key: '' for key in imrad_map}

    # Normalize and split text by lines
    lines = text.split('\n')
    current_section = None

    for line in lines:
        clean_line = line.strip().lower()
        for section, keywords in imrad_map.items():
            if any(kw in clean_line and len(clean_line) < 80 for kw in keywords):
                current_section = section
                break
        if current_section:
            sections[current_section] += line + '\n'

    return sections


In [16]:
output_data = []

folder_path = "valid_pdfs"

for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(folder_path, filename)
        try:
            text = extract_text(pdf_path)
            sections = extract_sections(text)
            output_data.append({
                "Filename": filename,
                "Introduction": sections['Introduction'],
                "Methodology": sections['Methodology'],
                "Results": sections['Results'],
                "Discussion": sections['Discussion']
            })
            print(f"✅ Processed: {filename}")
        except Exception as e:
            print(f"❌ Error in {filename}: {e}")


✅ Processed: 093_2024.acl-long.93.pdf
✅ Processed: 149_2024.acl-long.149.pdf
✅ Processed: 273_2024.acl-long.273.pdf


✅ Processed: 028_2024.acl-long.28.pdf


✅ Processed: 150_2024.acl-long.150.pdf


✅ Processed: 051_2024.acl-long.51.pdf
✅ Processed: 126_2024.acl-long.126.pdf
✅ Processed: 054_2024.acl-long.54.pdf
✅ Processed: 033_2024.acl-long.33.pdf
✅ Processed: 022_2024.acl-long.22.pdf
✅ Processed: 099_2024.acl-long.99.pdf
✅ Processed: 203_2024.acl-long.203.pdf
✅ Processed: 141_2024.acl-long.141.pdf
✅ Processed: 296_2024.acl-long.296.pdf
✅ Processed: 091_2024.acl-long.91.pdf


✅ Processed: 106_2024.acl-long.106.pdf
✅ Processed: 066_2024.acl-long.66.pdf
✅ Processed: 218_2024.acl-long.218.pdf
✅ Processed: 281_2024.acl-long.281.pdf
✅ Processed: 185_2024.acl-long.185.pdf
✅ Processed: 143_2024.acl-long.143.pdf
✅ Processed: 096_2024.acl-long.96.pdf
✅ Processed: 006_2024.acl-long.6.pdf
✅ Processed: 016_2024.acl-long.16.pdf


...

✅ Processed: 242_2024.acl-long.242.pdf
✅ Processed: 152_2024.acl-long.152.pdf
✅ Processed: 115_2024.acl-long.115.pdf
✅ Processed: 227_2024.acl-long.227.pdf


✅ Processed: 011_2024.acl-long.11.pdf
✅ Processed: 167_2024.acl-long.167.pdf
✅ Processed: 075_2024.acl-long.75.pdf
✅ Processed: 204_2024.acl-long.204.pdf
✅ Processed: 058_2024.acl-long.58.pdf
✅ Processed: 023_2024.acl-long.23.pdf
✅ Processed: 278_2024.acl-long.278.pdf
✅ Processed: 223_2024.acl-long.223.pdf
✅ Processed: 199_2024.acl-long.199.pdf
✅ Processed: 179_2024.acl-long.179.pdf
✅ Processed: 268_2024.acl-long.268.pdf


✅ Processed: 228_2024.acl-long.228.pdf


✅ Processed: 094_2024.acl-long.94.pdf
✅ Processed: 095_2024.acl-long.95.pdf
✅ Processed: 188_2024.acl-long.188.pdf


✅ Processed: 135_2024.acl-long.135.pdf
✅ Processed: 097_2024.acl-long.97.pdf
✅ Processed: 209_2024.acl-long.209.pdf
✅ Processed: 123_2024.acl-long.123.pdf
✅ Processed: 263_2024.acl-long.263.pdf
✅ Processed: 055_2024.acl-long.55.pdf


In [17]:
df = pd.DataFrame(output_data)
df.to_csv("sectionwise_dataset.csv", index=False, encoding='utf-8')
print("📁 CSV file 'sectionwise_dataset.csv' saved.")


📁 CSV file 'sectionwise_dataset.csv' saved.


In [18]:
from google.colab import files
files.download("sectionwise_dataset.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>